# This Jupyter Notebook goes through the different development stages of the Python Fullstack Assessment Project for Stori

## First steps:

* ### Work around CSV file and other data managment to create each message
* ### Work around Pythonic Email Delivery

## Secondary steps:

* ### Work around dockerization
* ### Work around extending the system to more than one users
* ### Work around AWS DevOps

## CSV file managment

### Using pandas and its built-in datetime utilities, the ```.csv``` files can be handled and processed easily. 

In [1]:
import pandas as pd

def csv2Summary(trxns_file):
    '''
        input =>    
            trxns_file: String
        output =>   
            trxns: Pandas DataFrame
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_sumarry: Sting
                
    '''
    #Total Balance
    trxns = pd.read_csv("trxn.csv")
    total_balance = trxns.transaction.sum() # sum of transactions
    #Debit and Credit Averages
    trxns["DebitorCredit"] = trxns.transaction.map( lambda x: 'credit' if x > 0 else 'debit')
    avg_credit = trxns.loc[trxns.DebitorCredit == "credit", 'transaction'].mean()
    avg_debit = trxns.loc[trxns.DebitorCredit == "debit", 'transaction'].mean()
    #Monthly Summaries
    trxns.date = pd.to_datetime(trxns.date, format = "%d/%m")
    trxns['month'] = trxns.date.dt.strftime('%B')
    trx_per_month = trxns.month.value_counts()
    monthly_summary = ""
    for month, count in zip(trx_per_month.index, trx_per_month):
        monthly_summary += f"<br>Number of transactions {month}: {count}<br>\n"
    
    return trxns, total_balance, avg_credit, avg_debit, monthly_summary

## Pythonic Email Delivery

### Using smtplib, SSL and Gmail, emails will be sent to a given account
### The authenticaton credentials for the sender will be stored in a pickle file, which only has to be created once. 

## Mailing Tools and Generic Email Body.

![email_1try.png](email_1try.png)

## Add Stori Logo:

### Follow Signarture and Logo suggestions from [Exclaimer](https://www.exclaimer.com/email-signature-handbook/using-the-right-email-signature-size/#:~:text=Email%20signature%20size%20dimensions,70%20to%20100%20pixels%20high.)

![stori.jpg](stori.jpg)
                                                                                                            

## Pickle File for authentication credentials

### Fill the credentials dictionary, run the code once, and then erase the entries.

### This code will create a ```.pkl``` file which will contain the mailing credentials. If the file already exists, then it will load it. 

### This way, sharing code is possible without revealing personal credentials;  ```.gitignore``` already includes this ```.pkl``` file, so it's not passed onto the GitHUB repo. 

In [2]:
import pickle
import os
if not os.path.exists('secret_credentials.pkl'):
    credentials={}
    credentials['Sender Email'] = "" # sender email account
    credentials['Sender Password'] = "" # sender email account password
    credentials['Receiver Email'] = "" # receiver email account
    with open('secret_credentials.pkl','wb') as f:
        pickle.dump(credentials, f)
else:
    credentials=pickle.load(open('secret_credentials.pkl','rb'))

In [3]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

def createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary):
    '''
        Input =>
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_summary: String
        Output=>
            message => SMTP.MIMEMultipart Message Object
    '''
    
    message = MIMEMultipart("alternative")

    
    html = f"""
    <html>
        <body>
            <div>
                <p>
                    Hello,<br>Herein is your Transactions Summary<br>
                </p>
            </div>
            <div>
                <table style="width:100%">
                      <tr>
                        <th>
                            <p>
                                <br>Total Balance: {total_balance}<br>
                                <br>Average Debit Amount: {avg_debit}<br>
                                <br>Average Credit Amount: {avg_credit}<br>
                            </p>
                        </th>
                        <th>
                            <p>
                                {monthly_summary}
                            </p>
                        </th>
                      </tr>
                </table>
            </div>  
            <div>
                <br>
                    <img align="left" width="300" height="100" src="cid:image1">
                <br>
            </div>
        </body>

    </html>
    """
    html_text = MIMEText(html, "html")
    message.attach(html_text)

    fp = open('stori.jpg', 'rb')
    msgImage = MIMEImage(fp.read())
    fp.close()

    msgImage.add_header('Content-ID', '<image1>')
    message.attach(msgImage)
    
    return message

def sendEmailSLL(credentials, receiver_email, subject, message):
    '''
        Input =>
            credentials: Dict
            receiver_email: String
            subject: String
            message: String}
        Output =>
            None: print either success or exception
    '''
    
    sender_email = credentials['Sender Email']
    password =credentials['Sender Password']
    
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    
    try:
        port = 465 # Assign 465 port for the SMTP server (Gmail's requirements)
        context = ssl.create_default_context() 

        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(
                sender_email, 
                receiver_email, 
                message.as_string()
            )
        print("email sent successfully")
    except Exception as e:
        print("error sending email: ", e)

In [4]:
trxns, total_balance, avg_credit, avg_debit, monthly_summary = csv2Summary("trxn.csv")
message = createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary)
sendEmailSLL(credentials, credentials['Receiver Email'], "Transactions Summary", message)

email sent successfully
